# Advent of Code

Solutions for Advent of Code 2023 puzzles.

In [1]:
def check_answer(result, answer):
    assert result == answer, f"Result not equal to answer {answer}."

In [2]:
import re
import itertools
import math

## Day 1

In [3]:
def day1_1(answer=None):
    with open("day1.txt", "r") as f:
        data = f.readlines()
    digits = [re.findall("[0-9]", d) for d in data]
    result = sum([(int(d[0] + d[-1])) for d in digits])
    if answer:
        check_answer(result, answer)
    return result

In [4]:
day1_1(answer=54877)

54877

In [5]:
def day1_2(answer=None):
    with open("day1.txt", "r") as f:
        data = f.readlines()

    words = {
        "one": 1,
        "two": 2,
        "three": 3,
        "four": 4,
        "five": 5,
        "six": 6,
        "seven": 7,
        "eight": 8,
        "nine": 9,
    }
    special_cases = []
    # Special cases for combinations like 'eighthree'
    for a, b in itertools.permutations(list(words.keys()), 2):
        if a[-1] == b[0]:
            special_cases.append((a[:-1] + b, a + b))

    for text, replacement in special_cases:
        data = [d.replace(text, replacement) for d in data]

    digits = [
        re.findall("(one|two|three|four|five|six|seven|eight|nine|[0-9])", s)
        for s in data
    ]

    pure_digits = []
    for s in digits:
        values = [words.get(v, v) for v in s]
        pure_digits.append(values)

    result = sum([int(str(s[0]) + str(s[-1])) for s in pure_digits])
    if answer:
        check_answer(result, answer)

    return result

In [6]:
day1_2(answer=54100)

54100

## Day 2

In [7]:
def parser2(data):
    parsed_data = []
    for line in data:
        game = int(re.findall('Game ([0-9]+)', line)[0])
        subsets_raw = re.sub(r'Game ([0-9]+): ', '', line).split(";")
        subsets = []
        for subset in subsets_raw:
            extract_number = lambda s, color: int(re.findall(f'([0-9]+) {color}', s)[0]) if re.findall(f'([0-9]+) {color}', s) else 0
            red = extract_number(subset, 'red')
            blue = extract_number(subset, 'blue')
            green = extract_number(subset, 'green')

            subsets.append((red,blue,green))
        
        parsed_data.append((game, subsets))
    return parsed_data

In [8]:
def day2_1(answer=None):
    with open("day2.txt", "r") as f:
        data = f.readlines()
    games = parser2(data)
    
    bag = (12, 14, 13) # red, blue, green
    possible_games, impossible_games = set(), set()
    for game in games:
        n, subsets = game[0], game[1]
        possible_games.add(n)
        for subset in subsets:
            if subset[0] > bag[0] or subset[1] > bag[1] or subset[2] > bag[2]:
                impossible_games.add(n)
                break
    possible_games = possible_games - impossible_games
    result = sum(possible_games)
    if answer:
        check_answer(result, answer)
    
    return result

In [9]:
day2_1(answer=1931)

1931

In [10]:
def day2_2():
    with open("day2.txt", "r") as f:
        data = f.readlines()
    games = parser2(data)
    
    bag = (12, 14, 13) # red, blue, green
    result = 0
    for game in games:
        reds = max([s[0] for s in game[1]])
        blues = max([s[1] for s in game[1]])
        greens = max([s[2] for s in game[1]])
        result += reds * blues *greens
    return result

In [12]:
day2_2()

83105

## Day 3

In [13]:
def parser3(filter_ch=None):
    with open("day3.txt", "r") as f:
        data = f.read().splitlines() 
    lines = data
    arr = [] # each value is a tuple (idx, value, parent_number)
    
    numbers = []
    symbols = [] # symbol's positions
    
    for i, line in enumerate(lines):
        line_vector = []
        for j, ch in enumerate(line):
            line_vector.append(ch)
            if ch != '.' and not ch.isdigit():
                if filter_ch and ch == filter_ch:
                    symbols.append((i,j))
                else:
                    symbols.append((i,j))
        arr.append(line_vector)
    
    return arr, symbols

In [14]:
def day3_1():
    
    arr, symbols = parser3()
    
    part_numbers = []
    lx, ly = len(arr), len(arr[0])
    visited_cells = []
    
    def is_cell_allowed(row, col, lx=lx, ly=ly):
        return row >= 0 and row < lx and col >= 0 and col < ly
    
    for symbol in symbols:
        i, j = symbol[0], symbol[1]
        # navigate around the symbol
        nodes = [(-1,1), (0,1), (1,1), (1,0), (1, -1), (0, -1), (-1,-1), (-1,0)]
        for (di, dj) in nodes:
            if is_cell_allowed(row=i + di, col=j + dj):
                cell = arr[i + di][j + dj]
                if (i + di, j + dj) not in visited_cells and cell.isdigit():
                    part_number = cell
                    # look to the left
                    row, col = i + di, j + dj - 1
                    if is_cell_allowed(row, col):
                        previous = arr[row][col]
                        visited_cells.append((row,col))
                        while previous.isdigit():
                            part_number = previous + part_number
                            col -= 1
                            if is_cell_allowed(row, col):
                                previous = arr[row][col]
                                visited_cells.append((row,col))
                            else:
                                break
                    # look to the right
                    row, col = i + di, j + dj + 1
                    if is_cell_allowed(row, col):
                        next_cell = arr[row][col]
                        visited_cells.append((row,col))
                        while next_cell.isdigit():
                            part_number = part_number + next_cell
                            col += 1
                            if is_cell_allowed(row, col):
                                next_cell = arr[row][col]
                                visited_cells.append((row,col))
                            else:
                                break

                    part_numbers.append(part_number)
                    
    part_numbers = [int(v) for v in part_numbers]
    return sum(part_numbers)

In [15]:
day3_1()

525119

In [16]:
def day3_2():
    arr, symbols = parser3(filter_ch="*")  
    
    all_part_numbers = []
    lx, ly = len(arr), len(arr[0])
    visited_cells = []
    
    def is_cell_allowed(row, col, lx=lx, ly=ly):
        return row >= 0 and row < lx and col >= 0 and col < ly
    
    
    for symbol in symbols: # only gears
        i, j = symbol[0], symbol[1]
        part_numbers = []
        
        # navigate around the symbol
        nodes = [(-1,1), (0,1), (1,1), (1,0), (1, -1), (0, -1), (-1,-1), (-1,0)]
        for (di, dj) in nodes:
            if is_cell_allowed(row=i + di, col=j + dj):
                cell = arr[i + di][j + dj]
                if (i + di, j + dj) not in visited_cells and cell.isdigit():
                    part_number = cell
                    # look to the left
                    row, col = i + di, j + dj - 1
                    if is_cell_allowed(row, col):
                        previous = arr[row][col]
                        visited_cells.append((row,col))
                        while previous.isdigit():
                            part_number = previous + part_number
                            col -= 1
                            if is_cell_allowed(row, col):
                                previous = arr[row][col]
                                visited_cells.append((row,col))
                            else:
                                break
                                
                    # look to the right
                    row, col = i + di, j + dj + 1
                    if is_cell_allowed(row, col):
                        next_cell = arr[row][col]
                        visited_cells.append((row,col))
                        while next_cell.isdigit():
                            part_number = part_number + next_cell
                            col += 1
                            if is_cell_allowed(row, col):
                                next_cell = arr[row][col]
                                visited_cells.append((row,col))
                            else:
                                break

                    part_numbers.append(part_number)
        
        part_numbers = [int(v) for v in part_numbers]
        all_part_numbers.append(part_numbers)
                    
    # after all symbols, check for gears
    gear_ratios = []
    for symbol_numbers in all_part_numbers:
        if symbol_numbers and len(symbol_numbers) == 2:
            gear_ratios.append(math.prod(symbol_numbers))
    
    return sum(gear_ratios)

In [17]:
day3_2()

76504829

## Day 4

In [18]:
def day4_1():
    with open("day4.txt", "r") as f:
        data = f.read().splitlines()
    points = 0
    for d in data:
        winning = set(re.findall('([0-9\s]+) \|', d)[0].split())
        mine = set(re.findall('\| ([0-9\s]+)', d)[0].split())
        if mine.intersection(winning):
            points += 2 ** (len(mine.intersection(winning)) - 1)
            
    return points

In [19]:
day4_1()

21558

In [20]:
def day4_2():
    with open("day4.txt", "r") as f:
        cards = f.read().splitlines()
    
    deck = {} # dict with card number and number of copies
    cards = cards
    
    for card in cards:
        idx = int(re.findall('Card\s+([0-9]+):', card)[0])
        if idx not in deck:
            deck[idx] = 1
    
    for card in cards:
        idx = int(re.findall('Card\s+([0-9]+):', card)[0])
        copies = deck.get(idx)
        winning = set(re.findall('([0-9\s]+) \|', card)[0].split())
        mine = set(re.findall('\| ([0-9\s]+)', card)[0].split())
            
        matchs = len(mine.intersection(winning))
        
        # get copies
        for i in range(idx + 1, idx + matchs + 1, 1):
            if i not in deck:
                deck[i] = 1
            else:
                deck[i] += copies # each copy gives new copies

    return sum(deck.values())

In [21]:
day4_2()

10425665

## Day 5